In [ ]:
# This adds the path to import the development version (git repo) of NDI Python
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from ndi import Experiment, DaqSystem, FileNavigator
from ndi.daqreaders import MockReader
from ndi.database import FileSystem as Database, Query as Q

In [ ]:
db = Database('../test_exp')

In [ ]:
fn = FileNavigator(epoch_file_patterns=['.*\\.wav', '.*\\.txt'], 
                   metadata_file_pattern='.*\\.txt')

In [ ]:
ds = DaqSystem(name='myDaq1',
              file_navigator=fn,
              daq_reader=MockReader)
ds2 = DaqSystem(name='myDaq2',
               file_navigator=fn,
               daq_reader=MockReader)

In [ ]:
e = Experiment(name='myExperiment3', 
               daq_systems=[ds, ds2])

In [ ]:
db.add_experiment(e)

In [8]:
e.ctx

In [9]:
e2 = db.find_by_id(Experiment, e.id)

In [10]:
e2 == e

True

In [11]:
e2 == db.find(Experiment,Q('name') == 'myExperiment3')[0]

True

In [12]:
db.find(Experiment, Q('name') == 'myExperiment2')

[]

In [13]:
ds_results = db.find(DaqSystem, Q('experiment_id') == e.id)

In [14]:
print(ds_results)

[<ndi.daq_system.DaqSystem object at 0x10f3ac340>, <ndi.daq_system.DaqSystem object at 0x10f3aceb0>]


In [15]:
e.daq_systems[0] == ds_results[0]

False

In [16]:
db.find(Experiment)

In [17]:
db.delete_by_id(Experiment, e.id)

In [18]:
try:
    db.find_by_id(Experiment, e.id)
except FileNotFoundError:
    print('File not found')

File not found


In [19]:
db.delete_many(Experiment)

In [20]:
db.find(Experiment)

[]

In [21]:
try:
    db.add(ds)
except FileExistsError as e:
    print(e)

File '../test_exp/.ndi/daq_systems/6c39ed70565b415eae705f17dc538fad.dat' already exists


In [22]:
db.upsert(ds)

In [23]:
print(ds.name)

myDaq1


In [24]:
db.update_by_id(DaqSystem, ds.id, { 'name': 'daq2' })

In [25]:
ds2 = db.find_by_id(DaqSystem, ds.id)

In [26]:
print(ds2.name)

daq2


In [27]:
try:
    db.add('a')
except TypeError as e:
    print(e)

'a' is not an instance of an 'NDI_Object' child class.


In [28]:
db.drop_collection(Experiment)
try:
    db.find(Experiment)
except KeyError as e:
    print(e)

<class 'ndi.experiment.Experiment'>
